In [9]:
%run input_data.ipynb

In [10]:
import pyomo.environ as pe
import pyomo.opt as po

In [11]:
# Initialize Model
model = pe.ConcreteModel()

# Sets
model.F = pe.Set(initialize = factories ,ordered = False)                                   # set of all factories
model.T = pe.Set(initialize = transshipment_points ,ordered = False)                        # set of all transshipment points
model.C = pe.Set(initialize = customers ,ordered = False)                                   # set of all customers    
model.P = pe.Set(initialize = periods, ordered = False)                                     # set of all periods
model.W = pe.Set(initialize = warehouses, ordered = False)                                  # set of all warehouses

# Decision Variables
model.x = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeReals)                   # quantity transported directly from factory to customer                       
model.y = pe.Var(model.T, model.C , model.P, domain = pe.NonNegativeReals)                  # quantity transported from transshipment point to customer                 
model.q_bl = pe.Var(model.F, block_trans, model.P, domain = pe.NonNegativeReals)            # quantity transported by mode block train                
model.q_s_w = pe.Var(model.F, single_trans, model.P, domain = pe.NonNegativeReals)          # quantity transported by mode single wagon train            
model.q_ss = pe.Var(model.F, shortsea_trans, model.P, domain = pe.NonNegativeReals)         # quantity transported by mode shortsea shipping           
model.q_ba = pe.Var(model.F, barge_trans, model.P, domain = pe.NonNegativeReals)            # quantity transported by mode barge                    
model.td = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeIntegers)               # number of trucks via direct route from factory to customer     
model.tid = pe.Var(model.T, model.C, model.P, domain = pe.NonNegativeIntegers)              # number of trucks via indirect route from transshipment point to customer   

# #Additional variables 
model.inv_level = pe.Var(model.W, model.P, domain = pe.NonNegativeReals)                    # inventory level for each warehouse during each period

In [12]:
# Objective function
objExpr = sum(sum(dist_fact_cust[i, j] * transfer_cost * model.x[i,j,p] for i in model.F for j in model.C)      \
        + sum(block_train_cost[i, j] * model.q_bl[i,j,p] for i in model.F for j in block_trans)                 \
        + sum(single_wagon_cost[i, j] * model.q_s_w[i,j,p] for i in model.F for j in single_trans)              \
        + sum(shortsea_cost[i, j] * model.q_ss[i,j,p] for i in model.F for j in shortsea_trans)                 \
        + sum(barge_cost[i, j] * model.q_ba[i,j,p] for i in model.F for j in barge_trans)                       \
        + sum(dist_point_cust[i, j] * transfer_cost * model.y[i,j,p] for i in model.T for j in model.C)         \
        + sum(handling_costs[i] * model.y[i,j,p] for i in model.T for j in model.C)                             \
        + sum(model.td[i,j,p] * truck_fixed_cost for i in model.F for j in model.C)                             \
        + sum(model.tid[i,j,p] * truck_fixed_cost for i in model.T for j in model.C) for p in model.P)          \

model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [13]:
# Contraints

# for each period, for each combination of factory and customer, the number of direct trucks >= the number of direct tonnes / truck capacity
model.trucksDirect = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        for j in model.C:
            expression = model.td[i,j,p] >= model.x[i,j,p] / truck_cap
            model.trucksDirect.add(expression)

# for each period, for each combination of factory and customer, the number of indirect trucks >= the number of indirect tonnes / truck capacity
model.trucksIndirect = pe.ConstraintList()
for p in model.P:
    for i in model.T:
        for j in model.C:
            expression = model.tid[i,j,p] >= model.y[i,j,p] / truck_cap
            model.trucksIndirect.add(expression)

# for each period, for each factory, the total number of tonnes transported from the factory should not exceed its production capacity
model.prodCap = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        expression = sum(model.x[i,j,p] for j in model.C) + sum(model.q_bl[i,j,p] for j in block_trans) + sum(model.q_s_w[i,j,p] for j in single_trans) + sum(model.q_ss[i,j,p] for j in shortsea_trans) + sum(model.q_ba[i,j,p] for j in barge_trans) <= production_cap[i]
        model.prodCap.add(expression)

# for each period, for each customer, the demands should be met taking into account the inventory level
model.meetDemand = pe.ConstraintList()
for p in model.P:
    for j in model.C:
        expression = sum(model.x[i,j,p] for i in model.F) + sum(model.y[i,j,p] for i in model.T) >= demands[p][j] - model.inv_level[j,p]
        model.meetDemand.add(expression)

# for each period, for each transshipment point, the number of tonnes coming in should equal the number of tonnes going out
model.flowBalance = pe.ConstraintList()
for p in model.P:
    total_quantity_trans = {"Robert Schmidtz": 0, "Wiechers": 0, "Haeger und Schmidt": 0, "Neska/UCT Neuss": 0, "WTA": 0, "Siefert spedition": 0, "Rhenus logistics": 0}
    for j in block_trans:
        expression_bl = sum(model.q_bl[i,j,p] for i in model.F)
        expression_s_w = sum(model.q_s_w[i,j,p] for i in model.F) 
        total_train = expression_bl + expression_s_w
        total_quantity_trans[j] += total_train
    for j in shortsea_trans:
        expression_ss = sum(model.q_ss[i,j,p] for i in model.F) 
        total_quantity_trans[j] += expression_ss
    for j in barge_trans:
        expression_ba = sum(model.q_ba[i,j,p] for i in model.F) 
        total_quantity_trans[j] += expression_ba
    for j in model.T: 
        expression = total_quantity_trans[j] == sum(model.y[j,k,p] for k in model.C)
        model.flowBalance.add(expression)

# for each warehouse, the inventory level of the first period equals the sum of the initial inventory level and all quantities coming in minus the demand of period 1
model.inventory = pe.ConstraintList()
for w in model.W:
    expression = model.inv_level[w,1] == InvLev[w] + sum(model.x[i,w,1] for i in model.F) + sum(model.y[i,w,1] for i in model.T) - demands[1][w] 
    model.inventory.add(expression)  

# for each period starting at period 2, for each warehouse, the inventory level equals the sum of the previous inventory level and all quantities coming in minus the demand of that period
model.update_inventory = pe.ConstraintList()
for p in model.P:
    for w in model.W: 
        if p > 1:
            expression = model.inv_level[w,p] == model.inv_level[w,p-1] + sum(model.x[i,w,p] for i in model.F) + sum(model.y[i,w,p] for i in model.T) - demands[p][w]
            model.update_inventory.add(expression)

# for each period, for each warehouse, the inventory level <= inventory capacity
model.inventory_cap = pe.ConstraintList()
for p in model.P:
    for w in model.W:
        expression = model.inv_level[w,p] <= InvCap[w]
        model.inventory_cap.add(expression)

In [14]:
# Solve with use of gurobi
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True, timelimit=300)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-22
Read LP format model from file /var/folders/8g/rd_mq8dj1jb8p1ql_31qtvgc0000gn/T/tmpbwu9rma2.pyomo.lp
Reading time = 0.02 seconds
x1951: 1141 rows, 1951 columns, 4853 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1141 rows, 1951 columns and 4853 nonzeros
Model fingerprint: 0x8b273a72
Variable types: 1151 continuous, 800 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [2e+00, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+03]
Found heuristic solution: objective 4.585867e+07
Presolve removed 89 rows and 61 columns
Presolve time: 0.22s
Presolved: 1052 rows, 1890 columns, 3916 nonzeros
Found heuristic solution: objective 4.584717e+07
Variable types: 1090 continuous, 800 integer (0 binary)

Root relaxation: objec

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/Faywestendorp/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/Faywestendorp/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Faywestendorp/anaconda3/lib/python3.7/site-packages/pyomo/common/tee.py", line 506, in _mergedReader
    list(handles), noop, noop, _poll)[0]
OSError: [Errno 9] Bad file descriptor



TimeoutExpired: Command '['/usr/local/bin/gurobi.sh']' timed out after 303.0 seconds